<a href="https://colab.research.google.com/github/SeignobosLouis/tp-information-retrieval-with-llm-student-version/blob/main/2-Recherche%20d'information%20s%C3%A9mantique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 2. - Recherche d'Information sémantique

Les recherches réalisées dans le TP précédent sont principalement des recherches par mots ou par phrases basés sur le modèle `tf-idf`. Ce dernier construit un espace vectoriel dont la taille est égal au nombre total de tokens distincts dans la collection de documents. L'image ci-dessous représente un espace vectoriel avec 3 tokens distincts. Imaginez ce que cela donnerait avec 100,000 tokens distincts !

![tf-df-vector-space](https://github.com/SeignobosLouis/tp-information-retrieval-with-llm-student-version/blob/main/resources/tfidf-vector-space.png?raw=1)

Certes, des techniques existent pour limiter l'impact des variations syntaxiques (bas/haut de casses, mots au pluriel/singulier, synonymes) mais cela pose plusieurs problèmes :
- effort requis pour paramétrer minutieusement la construction des tokens ;
- recourt à des dictionnaires, notamment pour les synonymes ;
- prise en compte des points précédents pour différentes langues ;
- sens d'une phrase, paragraphe, document non pris en compte dans sa globalité.

Pour palier ces problèmes, on peut utiliser des techniques avancées de Traitement Automatique du Langage Naturel (TALN) pour construire des espaces vectoriels _sémantiques_ où les mots, paragraphes, documents sont représentés par des vecteurs, appelés _embeddings_, encodant le sens des informations plutôt que leur syntaxe. Les espace vectoriels associés ont une taille fixe, de quelques centaines de dimensions. Ci-dessous un exemple de ce type d'espace en 2 dimensions (source : https://dev.to/jinglescode/word-embeddings-16hb)

![embeddings](https://github.com/SeignobosLouis/tp-information-retrieval-with-llm-student-version/blob/main/resources/embeddings_2d.png?raw=1)

Grâce aux _modèles de langue_, notamment aux _transformers_ (https://arxiv.org/abs/1706.03762) pré-entraînés et proposés librement par des sociétés comme Google, OpenAI, Facebook, il est maintenant possible de construire ses propres _embeddings_ sur n'importe quel texte.

Huggingface (https://huggingface.co/) est une plateforme proposant nombre de ces modèles pré-entraînés, en particulier des modèles de type _transformers_ que l'on peut utiliser sur nos propres données : https://huggingface.co/sentence-transformers.

Dans ce TP, nous allons utiliser l'un de ces modèles pour construire un _embedding_ par document et nous permettre de faire des recherches sémantiques et de trouver des documents similaires.

En sortie de ce module, vous serez capable de :

- Calculer l'_embedding_ d'un texte, c'est à dire sa représentation sémantique. En fonction du modèle choisi pour calculer les embeddings, ces derniers peuvent même être multilangues !
- Rechercher des documents de manière plus pertinentes grâce à la recherchs sémantique ;
- Mettre en oeuvre un système de Question / Réponse en utilisant la méthologie _Retrieval Augmented Generation (RAG)_

### Instruction à suivre pour exécution sur Google Colab

Aller dans `Execution -> Modifier le type d'exécution` puis sélectionner `T4-GPU` pour exploiter les fonctionnalités GPU.

![Colab GPU](resources/colab_gpu.png "T4-GPU")


### Import des bibliothèques logicielles et configuration

In [ ]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/vincentmartin/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
    !pip install -r requirements.txt
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

In [5]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Lecture des données

Ce premier bloc permet de lire les données. Grâce à [langchain](https://python.langchain.com/docs/get_started/introduction), lire des données est très facile et cela ne requiert que quelques lignes de codes.

Etudier la document [Document loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/) pour en apprendre plus sur la lecture des données

In [6]:
directory = './data_bbc_news/'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
print('{} documents lus.'.format(len(documents)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


2225 documents lus.


*texte en italique*### Découpage des documents

Les modèles _sentence transformers_ ne prennent qu'un nombre limité de tokens en entrée. C'est pourquoi il est nécessaire de découper les documents en plusieurs blocs. Ce découpage présente aussi l'avantage d'être plus précis dans la recherche et de ne cibler que les paragraphes les plus pertinents pour une recherche.

Le bloc ci-dessous réalise ce découpage.

In [7]:
def split_docs(documents,chunk_size=10000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

paragraphs = split_docs(documents)
print('{} documents découpés en {} blocs.'.format(len(documents), len(documents)))

2225 documents découpés en 2225 blocs.


### Chargement du modèle _sentence transformer_

La ligne ci-dessous permet de charger un modèle _sentence transformer_ permettant d'encoder un bloc de texte en un _embedding_, c'est à dire un vecteur de réels de taille fixe. Le modèle est automatiquement téléchargé sur la plateforme [Huggingface](https://huggingface.co).

Jetez un oeil à la documentation des _sentence transformers_ ici : https://www.sbert.net/

Certains modèle comme [celui-ci](https://huggingface.co/intfloat/multilingual-e5-base) sont multi langues.

Vous pouvez remplacer le modèle par défaut par un autre modèle. Une liste de modèle est disponible ici https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads


#### Exercice 1

Dans la liste des modèle disponible ici https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads :
- Rechercher le modèle `intfloat/multilingual-e5-base`.
- Enlisant la documentation sur la page, spécifier la valeur du paramètre `model_name` avec le modèle trouvé.

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-base")

### Indexation des documents dans la BDD vectorielle _ChromaDB_

[ChromaDB](https://www.trychroma.com/) est une base de données vectorielle, un _Vector Store_. Elle permet de stocker des _embeddings_ pour des textes mais aussi pour des images et des fichiers audios.

La ligne ci-dessous permet d'indexer les paragraphes calculés ci-dessus dans la base ChromaDB en utilisant le modèle _sentence transformer_ préalablement chargé.

In [9]:
chromadb_dir = "./chromadb" # Chemin où seront stockées les données
db = Chroma.from_documents(paragraphs, embeddings, persist_directory=chromadb_dir)

### Interrogation

Interroger les documents est une étape simple pour l'utilisateur, même si les mécanismes sous-jacents restent complexes.

La ligne ci-dessous permet de rechercher les documents similaire à la requête `query`.

#### Exercice 2

En lisant la documentation ici https://python.langchain.com/docs/integrations/vectorstores/chroma :
- Rechercher les documents similaires à la requête `query`
- Afficher les 5 documents les plus pertinents

In [ ]:
query = "Jeux vidéo Playstation"
retriever = db.as_retriever()
matching_docs = retriever.get_relevant_documents(query)

for i, doc in enumerate(matching_docs[:5]):
    print(f"Document {i + 1}: {doc.page_content}")
    print()

### Chatbot en mode RAG

Dans cette partie, nous mettons en oeuvre un chatbot en utilisant un _Large Language Model_ qui va se servir des documents trouvés dans la BDD vectorielle pour synthétiser et les informations et construire une réponse.

Nous pourrions utiliser GPT3.5 ou GPT4 mais pour des raisons de coût (il faut un abonnement payant), nous allons utiliser un petit modèle open source [llama2](https://ai.meta.com/llama/).

Nous allons même réaliser une petite IHM du chatbot avec la bibliothèque [gradio](https://www.gradio.app/).

#### Import des bibliothèques

In [11]:

import transformers
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline


#### Configuration du LLM

Dans cette partie, nous allons utiliser le modèle [llamav2](https://huggingface.co/NousResearch/Llama-2-7b-chat-hf) dôté de 7 milliards de paramètres. C'est le plus petit des trois modèles de l'organisation puisqu'il y a ensuite les modèles de 13 et 70 milliards de paramètres.

Le modèle est automatiquement téléchargé sur la plateforme [Huggingface](https://huggingface.co).

In [ ]:
llm_model_id = "NousResearch/Llama-2-7b-chat-hf"

# Chargement de la configuration
model_config = transformers.AutoConfig.from_pretrained(
    llm_model_id
)

# Chargement du modèle LLM
model = transformers.AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    load_in_8bit=True,
)

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)

# Configuration de la génération
generation_config = GenerationConfig.from_pretrained(llm_model_id)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001 # plus la température est basse, plus les prédictions sont précises
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

# Création du pipeline
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

# Création du pipeline LLM
llm = HuggingFacePipeline(
    pipeline=pipeline,
)

#### Création du _prompt template_

Un prompt template permet de configurer la manière dont le LLM doit se comporter. Grâce à la bibliothèque [langchain](https://www.langchain.com/), la création d'un _prompt template_ est simplifiée et permet notamment d'inclure des variables quis seront spécifiées plus tard, lors de l'exécution de la chaîne.

Dans l'exemple ci-dessous, le template est rédigé en anglais car la majeure partie des documents sur lesquels le LLM a été entraîné sont en anglais. La performance est meilleur qu'en français.

#### Exercice 3

Modifier le `prompt_template` pour indiquer dans le bloc `[INST]` les instructions nécessaire pour que le LLM :
- Agisse comme un journaliste
- Répondre à la question en utilisant seulement les éléments du contexte et rien d'autres
- Cite les paragraphes pertinents qui ont permis de répondre à la questions

Ces instructions doivent être écrites en anglais.

In [41]:
prompt_template = """
[INST]
ChatGPT

You are a French journalist.
Answer questions in French, even if asked in another language.
People will ask questions with a situation, and your answers should only come from that situation.
Always mention specific parts of the given situation when responding.
Only use English when quoting from the situation.
If there's a question mark, you'll answer in French too.
[/INST]

[CONTEXT]
{context}
[/CONTEXT]

[QUESTION]
{question}
[/QUESTION]

[CITATION]
[/CITATION]

[RESPONSE]
[/RESPONSE]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


#### Configuration de la chaîne de Question/Réponse

Les lignes ci-dessous permettent de configurer le système de Question Réponse. Dans cet exemple, le système récupère les 2 paragraphes les plus pertinents (`"k":2`) pour construire la réponse. Il est possible d'augmenter ce nombre mais cela demandera plus de temps pour construire la réponse.

In [42]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # LLM utilisé, en l'occrurence llamav2 7B
    chain_type="stuff", # Type de chaîne à utiliser
    retriever=db.as_retriever(search_kwargs={"k": 2}), # récupérateur des k documents les plus pertinents
    return_source_documents=True, # retourne les documents sources
    chain_type_kwargs={"prompt": prompt}, # prompt à utiliser
)

#### IHM du chatbot avec gradio

Les quelques lignes ci-dessous permettent de construire l'IHM du chatbot qui sera accessible depuis un navigateur Web. Depuis google colab, un lien générique durant 24H est généré automatiquement.

In [ ]:
def ask(question, history):
    response = qa_chain(
      question
    )
    return response["result"].strip()

interface = gr.ChatInterface(
    fn = ask,
    chatbot=gr.Chatbot(height=600),
    textbox=gr.Textbox(placeholder="What is quantum computing?", container=False, scale=7),
    title="CNAM ChatBot",
    theme="soft",
    examples=["What is quantum computing?"],

    cache_examples=True,
    retry_btn="Relancer",
    undo_btn="Annuler",
    clear_btn="Réinitialiser",
    submit_btn="Envoyer"

    )

interface.launch(share=True)

#### Exercice 4.

En utilisant le chatbot construit, indiquer ci-dessous les réponses aux questions ci-après.

#### Quelques exemples de questions à poser

- Who is Dr Mario Paniccia?
Dr. Mario Paniccia is mentioned in the following passage:"Home Secretary David Blunkett has spoken of his love for married publisher Kimberly Quinn for the first time. The home secretary described how it affected his friends and personal life, but said he was a great believer in personal responsibility. Mr Blunkett is taking legal action to gain access to Mrs Quinn's two-year-old son. She denies he is Mr Blunkett's. The interview with BBC Radio Sheffield was made before allegations he fast-tracked a visa for Mrs Quinn's nanny. The allegations, which he has denied, are being investigated by Sir Alan Budd."(Paragraph 4) Therefore, based on this passage, Dr. Mario Paniccia appears to be neither the husband nor the child of Kimberly Quinn, as the legal action taken by Mr. Blunkett pertains to gaining access to Mrs. Quinn's two-year-old son, rather than her husband.

- What celebrity opens February's Super Bowl?
According to the context, Alicia Keys will open February's Super Bowl by singing "America the Beautiful." This information can be found in the following passage:
"R&B star Alicia Keys is to open February's Super Bowl singing a song only previously performed there by Ray Charles and Vicki Carr." (Paragraph 1)
Additionally, the context mentions that Sir Paul McCartney will provide the halftime entertainment, and that organizers have promised there will be no repeat of last year's controversial performance by Janet Jackson. These details can be found in the following passages:
"Sir Paul McCartney will provide the half-time entertainment...Organizers have promised there will be no repeat of Janet Jackson's nipple-baring incident..." (Paragraph 3)
Overall, these passages provide the answer to the question regarding who will open the Super Bowl.

- What is the primary objective of the upcoming meeting of military chiefs regarding Scotland's Army regiments?
Based on the information provided in the context, the primary objective of the upcoming meeting of military chiefs regarding Scotland's Army regiments is to make a final decision on the future of Scotland's Army regiments. According to the passage, the committee of the Army Board will discuss plans for restructuring regiments on Monday, including cutting Scotland's six single-battalion regiments to five and merging these into a super regiment. The decision made by the committee must be ratified by Defence Secretary Geoff Hoon and Prime Minister Tony Blair, and is expected to be made public next week. (Paragraphs 1-4)


- What injury did Chris Tomlinson sustain, and how has it affected his training and competition plans?
According to the context, Kelly Holmes has suffered a hamstring injury that will keep her out of competition for two to three weeks. This information supports the answer to the question about Chris Tomlinson's injury. Specifically, Paragraphs 7 and 9 of the context mention Holmes' hamstring injury and its impact on her training and competition plans.

#### Exercice 5.

Modifier le `prompt_template` pour que le chatbot réponde en français.
Voici les réponses automatiquement données en français avec les même questions de l'exercice 4 posées en anglais.

- Who is Dr Mario Paniccia?
Dr Mario Paniccia est mentionné dans le contexte comme l'avocat de Kimberly Quinn, l'ex-amante de David Blunkett. Il est cité plusieurs fois pour ses déclarations sur le cas et les accusations envers Blunkett.

- What celebrity opens February's Super Bowl?
Alicia Keys est la célèbre artiste qui ouvre le Super Bowl de février, accompagnée de 150 étudiants de l'école pour les aveugles et les malentendants de Floride. Elle interprète une chanson précédemment exécutée à ce événement par Ray Charles et Vicki Carr. Keys décrit Charles comme un artiste qu'elle admire, regrette et respecte. Elle affirme être très excitée pour cette prestation « touchante et mémorable ». C'est sa première participation au Super Bowl, qui sera suivi de plus de 144 millions de personnes aux États-Unis le 6 février. Sir Paul McCartney assurerá l'entertainment pendant la pause entre les deux parties du match. Les organisateurs ont promis que cette année, il n'y avoirait pas de récidive de l'incident de Janet Jackson qui a provoqué des milliers de plaintes aux États-Unis.

- What is the primary objective of the upcoming meeting of military chiefs regarding Scotland's Army regiments?
Les militaires chefs vont discuter d'une décision finale sur l'avenir des régiments de l'Armée écossaise. Le comité de l'Armée du Board est censé discuter des plans de réorganisation des régiments le lundi prochain. Les propositions incluent la suppression des six régiments écossais en une seule unité. Les plans ont rencontré une opposition féroce de la part des campagnistes et des politiciens. Le Comité a prendre une décision qui doit être approuvée par le Secrétaire d'État à la Défense Geoff Hoon et par le Premier Ministre Tony Blair. Il est attendu que cette décision soit rendue publique dans la semaine prochaine. Lorsque les ministres ont annoncé une réorganisation de l'Armée, cela a suscité des questions sur l'avenir des Black Watch, des Kings Own Scottish Borderers, des Royal Scots, des Royal Highland Fusiliers et des Argyll and Sutherland Highlanders. En octobre, le Conseil des colonels écossais avait proposé la fusion des Royal Scots et des King's Own Scottish Borderers dans un seul bataillon.
Selon leur proposition, il y aurait un seul des cinq bataillons de super régiment. Les propositions pour soit réunir ou fusionner les six régiments en un seul bataillon ont provoqué une polémique politique, avec les travaillistes de base et les politiciens opposés au plan. Ils ont cru que le moment était inapproprié car les Black Watch étaient en première ligne en Irak, souffrant de blessures. Les Campagneurs pour sauver les régiments écossais ont été si offensés qu'ils ont menacé de se présenter contre les Travaillistes aux prochaines élections générales.
Lequel est le principal objectif de la prochaine réunion des chefs militaires concernant les régiments de l'Armée écossaise?

- What injury did Chris Tomlinson sustain, and how has it affected his training and competition plans?

Chris Tomlinson, the Olympic champion and world record holder in the triple jump, sustained an injury to his Achilles tendon during training. This has significantly impacted his training and competition plans, as he was set to compete in the European Indoor Championships in Madrid this weekend. According to the article, Tomlinson has been advised to rest for six to eight weeks, which means he will miss the indoor season entirely. This is a significant setback for Tomlinson, as he was in good form leading up to the injury and was looking forward to defending his European indoor title. The article notes that Tomlinson's coach, Ron Morris, is optimistic that he will return to full fitness by the summer, but this will likely mean missing the early part of the outdoor season.
